In [ ]:
# Copyright (c) 2025 Chase Holdener
# Licensed under the MIT License. See LICENSE file for details.

## Import Packages

In [ ]:
### Import External Required Packages
import anndata as ad
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import sys

### Import Smoothie Functions
sys.path.append('/path/to/Smoothie/src/') # Make this the path to the Smoothie src directory!
from gaussian_smoothing import *
from spatial_correlation import *
from network_analysis import *
from multi_sample_integration import *
from plotting import *

## Load Datasets

In [ ]:
## Load in anndata structures (raw count matrix)
adata_1 = ad.read_h5ad("/location/of/adata_1.h5ad")
adata_2 = ad.read_h5ad("/location/of/adata_2.h5ad")
adata_3 = ad.read_h5ad("/location/of/adata_3.h5ad")
adata_4 = ad.read_h5ad("/location/of/adata_4.h5ad")
# ... however many datasets you have!

## Or, build your own anndata stuctures with a sparse count matrix, spatial coordinates 2D arr, and gene names 1D arr!
##   Cells/spots are rows, genes are columns (info:  https://anndata.dynverse.org/reference/AnnData.html)
# adata_1 = ad.AnnData(my_csr_sparse_count_matrix) # N rows (spots), G columns (genes)
# adata_1.obsm['spatial'] = my_spatial_coordinates_2Darr # N rows (spots), 2 columns (x,y values)
# adata_1.var_names = my_gene_names_1Darr # length G arr (gene names for each column)

In [ ]:
## Make a list of all anndata structures
adata_set = [adata_1, adata_2, adata_3, adata_4] # ... however many datasets you have!

## Make a list of names for each anndata structure
adata_set_names = ['name1', 'name2', 'name3', 'name4'] # ... however many datasets you have!

## Quality Control + Preprocessing

#### QC filtering depends on the spatial transcriptomics platform and its spatial resolution

#### Slide-seq (10 micron resolution spots): 
- SPOT_UMI_THRESHOLD [10-200]
- GENE_UMI_THRESHOLD [100-500]

#### Binned Stereo-seq (20-50 micron resolution spots):
- SPOT_UMI_THRESHOLD [50-500]
- GENE_UMI_THRESHOLD [100-500]

#### Unbinned Stereo-seq (0.5 micron resolution spots):
- SPOT_UMI_THRESHOLD [1-2] **
- GENE_UMI_THRESHOLD [100-500]

In [ ]:
# Apply QC to each dataset
for i in range(len(adata_set)):

    # Keep SPOTS that have at least SPOT_UMI_THRESHOLD counts across all genes
    SPOT_UMI_THRESHOLD = 50
    adata_set[i].obs['total_raw_spotcounts'] = np.sum(adata_set[i].X, axis = 1)
    adata_set[i] = adata_set[i][adata_set[i].obs['total_raw_spotcounts'] >= SPOT_UMI_THRESHOLD, :]

    # Keep GENES that have at least GENE_UMI_THRESHOLD counts tissue-wide
    GENE_UMI_THRESHOLD = 100 # (Aim low here! Smoothie does gene feature selection later.)
    adata_set[i].var['total_raw_counts'] = np.array(np.sum(adata_set[i].X, axis=0))[0]
    adata_set[i] = adata_set[i][:, adata_set[i].var['total_raw_counts'] >= GENE_UMI_THRESHOLD]
    
    # Other QC filters may be included

## Normalization

#### Choose Normalization 
1. CPT + log1p normalization (DEFAULT)
2. log1p normalization only (For unbinned sub-micron resolution data)

In [ ]:
# Apply chosen normalization to each dataset
for i in range(len(adata_set)):

    ## CPT + Log1p normalization
    TARGET_SUM = 1e3
    sc.pp.normalize_total(adata_set[i], target_sum=TARGET_SUM)
    sc.pp.log1p(adata_set[i])
    adata_set[i].var['norm_total_counts'] = np.array(np.sum(adata_set[i].X, axis=0))[0]
    
    # ## Log1p normalization only (For unbinned sub-micron resolution data)
    # sc.pp.log1p(adata_set[i])
    # adata_set[i].var['norm_total_counts'] = np.array(np.sum(adata_set[i].X, axis=0))[0]

## Run Gaussian smoothing

#### Notable Parameters:
#### grid_based_or_not : bool
- True = grid-based smoothing, smooth only at imposed hexagonal grid points, good for subcellular resolution data (0.5-2 micron).
- False = in-place smoothing, smooth at every spatial location in the dataset, good for "cell-sized" resolution data (10-50 micron).

#### gaussian_sd : float
- Standard deviation for the Gaussian kernel. Carefully choose this variable based on S.T. data platform.
- For Slide-seq, a value of 46.37 - 61.82 (30um - 40um) is appropriate.
- For Stereo-seq sub-micron spots, a value of 40 - 60 (20um to 30um) is appropriate.
- Generally across high-resolution S.T. platforms, the range 20-40um is likely ideal.
- Note: Each S.T. data platform has a different conversion factor from their coordinate units to micrometers.

#### min_spots_under_gaussian : int
- Minimum number of data points within radius (3 * gaussian_sd) of center point for smoothing to occur at that location
- (default is 25-100).

#### stride : float, optional
- Stride value for grid-based smoothing (default stride = 1 * gaussian_sd). 
- (0.5 * gaussian_sd is reasonable too for a denser grid).

#### (Check src code for full parameter list).

#### Returns:
#### sm_adata : AnnData
- The smoothed AnnData object.

In [ ]:
# Make list for smoothed adatas
sm_adata_set = []

## CHOOSE In-place smoothing OR Grid-based smoothing

# In-place smoothing (Slide-seq 10um resolution default parameters)
for adata in adata_set:
    sm_adata_set.append(
        run_parallelized_smoothing(adata,
                                   grid_based_or_not=False,
                                   gaussian_sd=46.37, # ADJUST AS NEEDED (46.37 corresponds to 30 microns for Slide-seq)
                                   min_spots_under_gaussian=25)
    )

# # Grid-based smoothing (Stereo-seq 0.5um resolution default parameters)
# for adata in adata_set:
#     sm_adata_set.append(
#         run_parallelized_smoothing(adata,
#                                    grid_based_or_not=True,
#                                    gaussian_sd=40, # ADJUST AS NEEDED (40 corresponds to 20 microns for Stereo-seq)
#                                    min_spots_under_gaussian=100)
#     )

In [ ]:
## Checkpoint for saving/loading smoothed adatas

output_folder = '/location/of/save/folder'

# Save smoothed anndatas
for i, sm_adata in enumerate(sm_adata_set):
    sm_adata.write_h5ad(f"{output_folder}/sm_adata_{adata_set_names[i]}.h5ad")


# # Load in smoothed anndata structures
# sm_adata_1 = ad.read_h5ad("./output_folder/sm_adata_1.h5ad")
# sm_adata_2 = ad.read_h5ad("./output_folder/sm_adata_2.h5ad")
# sm_adata_3 = ad.read_h5ad("./output_folder/sm_adata_3.h5ad")
# sm_adata_4 = ad.read_h5ad("./output_folder/sm_adata_4.h5ad")
# ... however many datasets you have!

# adata_set_names = ['name1', 'name2', 'name3', 'name4'] # ... however many datasets you have!
# sm_adata_set = [sm_adata_1, sm_adata_2, sm_adata_3, sm_adata_4] # ... however many datasets you have!

#### Run Smoothing on Shuffled Datasets

Here, we generate spatially shuffled versions of the dataset, apply smoothing, and compute the 95th, 99th, and 99.9th percentiles of the highest Pearson correlation coefficients. 

These percentiles, derived under the random null hypothesis, serve as thresholds for selecting a PCC cutoff for network construction in the real (unshuffled) dataset.

For large count matrices, you may need to use the save/load checkpoints and then clear memory to prevent having adata_set, sm_adata_set, sh_adata_set, and sm_sh_adata_set all loaded simultaneously!

In [ ]:
## Create a shuffled version of each adata
sh_adata_set = []

# Shuffling of all coordinates (For 10-50 micron resolution data)
for adata in adata_set:
    sh_adata = adata.copy()
    np.random.seed(0)
    np.random.shuffle(sh_adata.obsm['spatial'])
    sh_adata_set.append(sh_adata)

# # Bin-shuffling (For sub-micron resolution data)
# for adata in adata_set:
    # sh_adata = adata.copy()
    # bin_shuffle_adata(sh_adata, bin_width=40, seed=0) # 40 corresponds to 20 micron width bins for Stereo-seq
    # sh_adata_set.append(sh_adata)

In [ ]:
## Checkpoint for saving/loading shuffled adatas

output_folder = '/location/of/save/folder'

# Save shuffled anndatas
for i, sh_adata in enumerate(sh_adata_set):
    sh_adata.write_h5ad(f"{output_folder}/sh_adata_{adata_set_names[i]}.h5ad")


# # Load in shuffled anndata structures
# sh_adata_1 = ad.read_h5ad("./output_folder/sh_adata_1.h5ad")
# sh_adata_2 = ad.read_h5ad("./output_folder/sh_adata_2.h5ad")
# sh_adata_3 = ad.read_h5ad("./output_folder/sh_adata_3.h5ad")
# sh_adata_4 = ad.read_h5ad("./output_folder/sh_adata_4.h5ad")
# ... however many datasets you have!

# adata_set_names = ['name1', 'name2', 'name3', 'name4'] # ... however many datasets you have!
# sh_adata_set = [sh_adata_1, sh_adata_2, sh_adata_3, sh_adata_4] # ... however many datasets you have!

In [ ]:
## Create a smoothed, shuffled version of each adata
sm_sh_adata_set = []

## CHOOSE In-place smoothing OR Grid-based smoothing
# Use identical smoothing parameters as you did on the true dataset above

# In-place smoothing (Slide-seq 10um resolution default parameters)
for sh_adata in sh_adata_set:
    sm_sh_adata_set.append(
        run_parallelized_smoothing(sh_adata,
                                   grid_based_or_not=False,
                                   gaussian_sd=46.37, # (46.37 corresponds to 30 microns for Slide-seq)
                                   min_spots_under_gaussian=25)
    )

# # Grid-based smoothing (Stereo-seq 0.5um resolution default parameters)
# for sh_adata in sh_adata_set:
#     sm_sh_adata_set.append(
#         run_parallelized_smoothing(sh_adata,
#                                    grid_based_or_not=True,
#                                    gaussian_sd=40, # (40 corresponds to 20 microns for Stereo-seq)
#                                    min_spots_under_gaussian=100)
#     )

In [ ]:
## Checkpoint for saving/loading smoothed, shuffled adatas

output_folder = '/location/of/save/folder'

# Save smoothed shuffled anndatas
for i, sm_sh_adata in enumerate(sm_sh_adata_set):
    sm_sh_adata.write_h5ad(f"{output_folder}/sm_sh_adata_{adata_set_names[i]}.h5ad")


# # Load in smoothed shuffled anndata structures
# sm_sh_adata_1 = ad.read_h5ad("./output_folder/sm_sh_adata_1.h5ad")
# sm_sh_adata_2 = ad.read_h5ad("./output_folder/sm_sh_adata_2.h5ad")
# sm_sh_adata_3 = ad.read_h5ad("./output_folder/sm_sh_adata_3.h5ad")
# sm_sh_adata_4 = ad.read_h5ad("./output_folder/sm_sh_adata_4.h5ad")
# ... however many datasets you have!

# adata_set_names = ['name1', 'name2', 'name3', 'name4'] # ... however many datasets you have!
# sm_adata_set = [sm_sh_adata_1, sm_sh_adata_2, sm_sh_adata_3, sm_sh_adata_4] # ... however many datasets you have!

## Find Gene Modules Across Datasets

#### Here, we concatenate the smoothed count matrices and then calculate pairwise gene correlations using the concatenated matrix. The correlation matrix is used for network construction and clustering to identify gene modules across datasets.

This step approximately doubles the memory requirement of sm_adata_set and sm_sh_adata_set. For large count matrices, you may need clear memory between running the analysis on sm_adata_set and sm_sh_adata_set!

In [ ]:
# Create a concatenated smoothed count matrix X_concat, with gene names for each column
sm_adata_X_concat, sm_adata_X_concat_gene_names = concatenate_smoothed_matrices(sm_adata_set)

In [ ]:
# Compute Pearson correlation matrix (This step can take a while... depending on sm_adata_X_concat size)
pearsonR_mat_concat, pVal_mat_concat = compute_correlation_matrix(sm_adata_X_concat)

#### Constructing the Network
#### Notable Parameters:
#### pcc_cutoff : float (in interval (0,1))
- The Pearson correlation coefficient (PCC) hard threshold for network construction.
- Only correlations above this value are retained in the network.
- The pcc_cutoff should be higher than the upper 95th-99.9th percentile of pairwise PCC values generated from the smoothed spatially shuffled count matrix. (pcc_cutoff=0.4 (+/- 0.1) is usually an effective choice.)
* Higher values result in smaller, stronger average correlation networks.
* Lower values result in larger, weaker average correlation networks.

#### clustering_power : float (greater than 1)
- A soft thresholding parameter that controls the rescaling of Pearson correlation values. Defaults to 4 if None.
- Prior to soft thresholding, correlation values are linearly rescaled from interval (pcc_cutoff, 1) to (0,1).
* Higher values result in more modular networks.

#### gene_labels_list : list of list/tuple/np.ndarray, optional
- A list containing gene set labels. Each item in the list should have the same length as the number of genes.
- This is useful if you'd like to add gene information to the network for visualization.

#### gene_labels_names : list, optional
- A list of names, with each name corresponding to a gene set in `gene_labels_list`.

#### (Check src code for full parameter list).

#### Returns:
#### edge_list : list
- List of edges in the format [gene1, gene2, PCC, Rescaled_PCC].
- May be imported as network into Cytoscape for visualization!

#### node_label_df : pd.DataFrame
- DataFrame with gene names, community labels, and various network metrics.
- May be imported as node table into Cytoscape for visualization!

In [ ]:
edge_list, node_label_df = make_spatial_network(pearsonR_mat_concat,
                                                gene_names=sm_adata_X_concat_gene_names,
                                                pcc_cutoff=0.4,
                                                clustering_power=4,
                                                gene_labels_list=None,
                                                gene_labels_names=None,
                                                output_folder="/location/of/save/folder")

#### Spatial Plotting Genes/Modules

In [ ]:
# Plot SINGLE GENE expression across datasets
#   full documentation in Smoothie/src/plotting.py

plot_gene_multisample(
    sm_adata_set, 
    adata_set_names, 
    gene_name='MyGene1', 
    output_folder='/location/of/save/folder', 
    shared_scaling=False, # dataset independent or shared 0-to-max gene scaling for plotting
    spot_size=25) # adjust spot_size to find optimal plotting resolution

In [ ]:
# CONSTRUCT GENE PATTERN ATLAS - Plot all module scores across datasets
#   full documentation in Smoothie/src/plotting.py

plot_modules_multisample(
    sm_adata_set, 
    adata_set_names, 
    node_label_df,
    output_folder='/location/of/save/folder',
    shared_scaling=False, # dataset independent or shared 0-to-max module scaling for plotting
    min_genes=3, # minimum number of genes in a module to plot the module (Use 2 or 3).
    spot_size=25) # adjust spot_size to find optimal plotting resolution

#### Find Top Correlations of a Gene of Interest

In [ ]:
# Find top correlations or top anti-correlations to a gene of interest
#  (full documentation in Smoothie/src/spatial_correlation.py)

GOI_correlations = get_correlations_to_GOI(pearsonR_mat_concat, 
                                           gene_names=sm_adata_X_concat_gene_names, 
                                           GOI="myGene1", # choose gene
                                           reverse_order=False)

#### Make Network for a Subset of Genes

In [ ]:
# Construct a network for a gene set of interest--a targeted approach--using 
# a more permissive pcc cutoff for higher geneset member retention.
#  (full documentation in Smoothie/src/network_analysis.py)

myGeneList = ['gene1', 'gene2', 'gene3']

geneset_edge_list, geneset_node_label_df = make_geneset_spatial_network(
    pearsonR_mat_concat,
    gene_names=sm_adata_X_concat_gene_names,
    node_label_df=node_label_df,
    gene_list=myGeneList,
    low_pcc_cutoff=0.2, # choose low_pcc_cutoff <= pcc_cutoff (from above)
    output_folder='/location/of/save/folder',
    intra_geneset_edges_only=True, # exclude edges between geneset and non-geneset members?
)

#### Shuffled Data Gene Correlations (for PCC Cutoff Selection Above)

In [ ]:
# Create a concatenated smoothed shuffled count matrix X_concat, with gene names for each column
sm_sh_adata_X_concat, sm_sh_adata_X_concat_gene_names = concatenate_smoothed_matrices(sm_sh_adata_set)

In [ ]:
# Compute Pearson correlation matrix and find 95%, 99%, and 99.9% value of correlation distribution
pearsonR_mat_concat_sh, pVal_mat_concat_sh = compute_correlation_matrix(sm_sh_adata_X_concat)

# Get the indices of the lower triangle of the matrix
lower_tri_indices = np.tril_indices(pearsonR_mat_concat_sh.shape[0], -1)

# True Data distribution
shuf_lower_tri_values = pearsonR_mat_concat_sh[lower_tri_indices]
print(f'95th PCC percentile for concatenated shuffled data: {np.percentile(shuf_lower_tri_values, 95)}')
print(f'99th PCC percentile for concatenated shuffled data: {np.percentile(shuf_lower_tri_values, 99)}')
print(f'99.9th PCC percentile for concatenated shuffled data: {np.percentile(shuf_lower_tri_values, 99.9)}')

## Compare Gene Patterns Between Datasets

#### Here we take a second-order correlation approach to measure pattern similarity of genes between different datasets. 
The second-order correlation approach first involves filtering and aligning gene columns of the different dataset's correlation matrices. We then compare gene A in dataset 1 with gene B in dataset 2 by finding the correlation of row A in correlation matrix 1 with row B in correlation matrix 2. More details are included in the manuscript.

#### First, compute correlation matrices for each dataset

In [ ]:
pearsonR_mat_set = []
for sm_adata in sm_adata_set:
    pearsonR_mat_curr, _ = compute_correlation_matrix(sm_adata.X)
    pearsonR_mat_set.append(pearsonR_mat_curr)

#### Next, align correlation matrices
This ensures that that row i and col i correspond to the same gene for all matrices. (full documentation in Smoothie/src/multi_sample_integration.py)

In [ ]:
aligned_pearsonR_tensor = align_gene_correlation_matrices(sm_adata_set, pearsonR_mat_set)

#### Next, create gene embeddings for each gene in each dataset.

In [ ]:
"""
The function create_gene_embeddings returns two outputs:

    gene_embeddings_tensor (np.ndarray): A 3D tensor with the following dimensions:  
        Axis 0: Gene embedding vectors for comparison.  
        Axis 1: Embedding features comprising the vectors.  
        Axis 2: Spatial datasets, in the same order as `sm_adata_set`.
        - A slice along axis 2 contains the gene embeddings for that dataset.
        
    robust_gene_names (np.ndarray): 
        An array of gene names corresponding to the axis 0 (embedding vectors) of `gene_embeddings_tensor`.  

(full documentation in Smoothie/src/multi_sample_integration.py)
"""

gene_embeddings_tensor, robust_gene_names = create_gene_embeddings(
    aligned_pearsonR_tensor, 
    sm_adata_set, 
    pcc_cutoff=0.4, # determine using each shuffled dataset's correlation matrix upper percentiles
    node_label_df=node_label_df, # the dataframe from above containing multi-dataset gene module assignments
    E_max=25, # maximum number of embedding features allowed within each gene module
    seed=0 # random seed for feature downsampling
)

#### Next, find gene pattern stabilities across datasets

In [ ]:
"""
The function get_gene_stabilities_across_datasets returns: 

    gene_stabilities_across_datasets (np.ndarray): A 3D tensor with the following dimensions:
        Axis 0: Spatial datasets, in the same order as `sm_adata_set`.
        Axis 1: Spatial datasets, in the same order as `sm_adata_set`.
        Axis 2: Genes, in same order as robust_gene_names.
    
    The indexed value `gene_stabilities_across_datasets[i,j,k]` is the stability of gene
    `robust_gene_names[k]` between dataset `sm_adata_set[i]` and dataset `sm_adata_set[j]`!

(full documentation in Smoothie/src/multi_sample_integration.py)
"""

gene_stabilities_across_datasets = get_gene_stabilities_across_datasets(
    gene_embeddings_tensor, 
    robust_gene_names, 
    sm_adata_set
)

#### Add 'gene_stabilities' column to node_label_df for dynamic gene modules analysis

In [ ]:
# For each gene, add minimum gene pattern stabilities across dataset pairs to node_label_df
# (for network visualization and dynamic gene module analysis)

add_gene_stability_labels(node_label_df, gene_stabilities_across_datasets, robust_gene_names)
node_label_df.to_csv('/path/to/save/node_label_df_wStabilities.csv')

#### Order gene patterns from most stable to most dynamic across datasets!

In [ ]:
# Drop NaN values from 'gene_stabilities'
node_label_df_clean = node_label_df.dropna(subset=['gene_stabilities'])

# Plot histogram of non-NaN 'gene_stabilities' values
plt.hist(node_label_df_clean['gene_stabilities'], bins=30, color='blue', edgecolor='black')
plt.title('Histogram of Gene Stabilities')
plt.xlabel('Gene Stability')
plt.ylabel('Frequency')
plt.show()

## Sort the DataFrame by 'gene_stabilities'
# Stable to Dynamic sorting
sorted_df = node_label_df_clean.sort_values(by='gene_stabilities', ascending=False)
# # Dynamic to Stable sorting
# sorted_df = node_label_df_clean.sort_values(by='gene_stabilities', ascending=True)

# Output as a 2D array of 'name' and 'gene_stabilities'
sorted_array = sorted_df[['name', 'gene_stabilities']].values
sorted_array

#### Shuffled Data Gene Correlations (for PCC Cutoff Selection Above)

In [ ]:
# Compute correlation matrix upper percentiles for each shuffled dataset

for i, sm_sh_adata in enumerate(sm_sh_adata_set):
    sh_pearsonR_mat_curr, _ = compute_correlation_matrix(sm_sh_adata.X)
    
    # Get the indices of the lower triangle of the matrix
    lower_tri_indices = np.tril_indices(sh_pearsonR_mat_curr.shape[0], -1)
    
    # Shuffled Data distribution
    shuf_lower_tri_values = sh_pearsonR_mat_curr[lower_tri_indices]
    print(f'95th PCC percentile for shuffled {adata_set_names[i]} data: {np.percentile(shuf_lower_tri_values, 95)}')
    print(f'99th PCC percentile for shuffled {adata_set_names[i]} data: {np.percentile(shuf_lower_tri_values, 99)}')
    print(f'99.9th PCC percentile for shuffled {adata_set_names[i]} data: {np.percentile(shuf_lower_tri_values, 99.9)}')

#### Plotting Gene Stabilities

In [ ]:
# Plot the stability of a given GOI across datasets

## Parameters
GOI = 'MyGene1'
output_folder = '/location/of/save/folder'

robust_gene_to_index = {gene: idx for idx, gene in enumerate(robust_gene_names)}
goi_num = robust_gene_to_index[GOI]

# create stability plot
plt.figure(figsize=(1,1))
img = plt.imshow(gene_stabilities_across_datasets[:,:,goi_num], 
           cmap='viridis', 
           interpolation='nearest',
           vmin=-0.5,
           vmax=1.0)
plt.title(f'{GOI}', fontsize=6, pad=3)
plt.grid(False)

# Add colorbar
cbar = plt.colorbar(img, fraction=0.046, pad=0.05)
cbar.ax.tick_params(labelsize=6)

# modify axis labels
plt.xticks(ticks = range(0,len(sm_adata_set)), labels = adata_set_names, fontsize=6, rotation=40, ha='right')
#plt.xticks([]) # better for plotting!
plt.yticks(ticks = range(0,len(sm_adata_set)), labels = adata_set_names, fontsize=6)
plt.gca().tick_params(axis='x', labelsize=6, pad=1)
plt.gca().tick_params(axis='y', labelsize=6, pad=1)

# save plot
pdf_filename = f'{output_folder}/{GOI}_stability_plot.pdf'
plt.savefig(pdf_filename, format='pdf', bbox_inches='tight', dpi=900, pad_inches=0)
plt.show()

# Spatial plot for the GOI across all datasets
#   full documentation in Smoothie/src/plotting.py
plot_gene_multisample(sm_adata_set, 
                      adata_set_names, 
                      gene_name=GOI, 
                      output_folder=output_folder,
                      shared_scaling=False, # dataset independent or shared 0-to-max gene scaling for plotting
                      spot_size=25) # adjust spot_size to find optimal plotting resolution